<a href="https://colab.research.google.com/github/RogerUltimate/-Keras-study---project/blob/main/%EB%94%B0%EB%A6%89%EC%9D%B4_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Keras study/따릉이 예측/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Keras study/따릉이 예측/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Keras study/따릉이 예측/submission.csv')

In [ ]:
train.describe()

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
count,1459.000000,1459.000000,1457.000000,1457.000000,1450.000000,1457.000000,1457.000000,1383.000000,1369.000000,1342.000000,1459.000000
mean,1105.914325,11.493489,16.717433,0.031572,2.479034,52.231297,1405.216884,0.039149,57.168736,30.327124,108.563400
std,631.338681,6.922790,5.239150,0.174917,1.378265,20.370387,583.131708,0.019509,31.771019,14.713252,82.631733
min,3.000000,0.000000,3.100000,0.000000,0.000000,7.000000,78.000000,0.003000,9.000000,8.000000,1.000000
25%,555.500000,5.500000,12.800000,0.000000,1.400000,36.000000,879.000000,0.025500,36.000000,20.000000,37.000000
50%,1115.000000,11.000000,16.600000,0.000000,2.300000,51.000000,1577.000000,0.039000,51.000000,26.000000,96.000000
75%,1651.000000,17.500000,20.100000,0.000000,3.400000,69.000000,1994.000000,0.052000,69.000000,37.000000,150.000000
max,2179.000000,23.000000,30.000000,1.000000,8.000000,99.000000,2000.000000,0.125000,269.000000,90.000000,431.000000


In [ ]:
test

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5
0,0,7,20.7,0.0,1.3,62.0,954.0,0.041,44.0,27.0
1,1,17,30.0,0.0,5.4,33.0,1590.0,0.061,49.0,36.0
2,2,13,19.0,1.0,2.1,95.0,193.0,0.020,36.0,28.0
3,4,6,22.5,0.0,2.5,60.0,1185.0,0.027,52.0,38.0
4,5,22,14.6,1.0,3.4,93.0,218.0,0.041,18.0,15.0
...,...,...,...,...,...,...,...,...,...,...
710,2148,1,24.6,0.0,2.4,60.0,1745.0,NaN,NaN,NaN
711,2149,1,18.1,0.0,1.0,55.0,2000.0,NaN,NaN,NaN
712,2165,9,23.3,0.0,2.3,66.0,1789.0,0.020,17.0,15.0
713,2166,16,27.0,0.0,1.6,46.0,1956.0,0.032,40.0,26.0


In [ ]:
submission

,id,count
0,0,NaN
1,1,NaN
2,2,NaN
3,4,NaN
4,5,NaN
...,...,...
710,2148,NaN
711,2149,NaN
712,2165,NaN
713,2166,NaN


In [ ]:
cols = train.columns
cols

Index(['id', 'hour', 'hour_bef_temperature', 'hour_bef_precipitation',
       'hour_bef_windspeed', 'hour_bef_humidity', 'hour_bef_visibility',
       'hour_bef_ozone', 'hour_bef_pm10', 'hour_bef_pm2.5', 'count'],
      dtype='object')

In [ ]:
def make_sequene_dataset(feature, label, window_size) :
    feature_list = []
    label_list = []

    for i in range(len(feature)-window_size) :
        feature_list.append(feature[i:i+window_size])
        label_list.append(label[i+window_size])

    return np.array(feature_list), np.array(label_list)


In [ ]:
window_size = 1

In [ ]:
scaler = MinMaxScaler()
scale_cols = ['hour', 'hour_bef_temperature','hour_bef_windspeed','hour_bef_ozone','count']
scaled_df = scaler.fit_transform(train[scale_cols])
scaled_df = pd.DataFrame(scaled_df, columns=scale_cols)

feature_cols = ['hour', 'hour_bef_temperature','hour_bef_windspeed','hour_bef_ozone']
label_cols = ['count']
feature_df = pd.DataFrame(scaled_df, columns=feature_cols)
label_df = pd.DataFrame(scaled_df, columns=label_cols)
feature_np = feature_df.to_numpy()
label_np = label_df.to_numpy()

In [ ]:
scaled_df

,hour,hour_bef_temperature,hour_bef_windspeed,hour_bef_ozone,count
0,0.869565,0.490706,0.1875,0.196721,0.111628
1,0.565217,0.631970,0.1750,0.319672,0.367442
2,0.260870,0.401487,0.0875,0.245902,0.058140
3,1.000000,0.185874,0.3375,0.303279,0.130233
4,0.782609,0.981413,0.6000,0.442623,1.000000
...,...,...,...,...,...
1454,0.173913,0.509294,0.2000,0.229508,0.046512
1455,0.130435,0.286245,0.4750,0.295082,0.044186
1456,0.217391,0.565056,0.2375,0.049180,0.048837
1457,0.913043,0.654275,0.4625,0.647541,0.500000


In [ ]:
feature_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   hour                  1459 non-null   float64
 1   hour_bef_temperature  1457 non-null   float64
 2   hour_bef_windspeed    1450 non-null   float64
 3   hour_bef_ozone        1383 non-null   float64
dtypes: float64(4)
memory usage: 45.7 KB


In [ ]:
label_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   count   1459 non-null   float64
dtypes: float64(1)
memory usage: 11.5 KB


In [ ]:
feature_df.describe()

,hour,hour_bef_temperature,hour_bef_windspeed,hour_bef_ozone
count,1459.000000,1457.000000,1450.000000,1383.000000
mean,0.499717,0.506224,0.309879,0.296303
std,0.300991,0.194764,0.172283,0.159914
min,0.000000,0.000000,0.000000,0.000000
25%,0.239130,0.360595,0.175000,0.184426
50%,0.478261,0.501859,0.287500,0.295082
75%,0.760870,0.631970,0.425000,0.401639
max,1.000000,1.000000,1.000000,1.000000


In [ ]:
feature_np

array([[0.86956522, 0.49070632, 0.1875    , 0.19672131],
       [0.56521739, 0.63197026, 0.175     , 0.31967213],
       [0.26086957, 0.40148699, 0.0875    , 0.24590164],
       ...,
       [0.2173913 , 0.56505576, 0.2375    , 0.04918033],
       [0.91304348, 0.65427509, 0.4625    , 0.64754098],
       [0.73913043, 0.66914498, 0.3875    , 0.35245902]])

In [ ]:
label_np

array([[0.11162791],
       [0.36744186],
       [0.05813953],
       ...,
       [0.04883721],
       [0.5       ],
       [0.39302326]])

In [ ]:
X, Y = make_sequene_dataset(feature_np, label_np, window_size)
x_train = X[:-200]
y_train = Y[:-200]
x_val = X[-200:]
y_val = Y[-200:]


In [ ]:
print(x_train.shape)
print(y_train.shape)

(1254, 5, 4)
(1254, 1)


In [ ]:
print(x_val.shape)
print(y_val.shape)

(200, 5, 4)
(200, 1)


In [ ]:
x_train

array([[[0.86956522, 0.49070632, 0.1875    , 0.19672131],
        [0.56521739, 0.63197026, 0.175     , 0.31967213],
        [0.26086957, 0.40148699, 0.0875    , 0.24590164],
        [1.        , 0.18587361, 0.3375    , 0.30327869],
        [0.7826087 , 0.98141264, 0.6       , 0.44262295]],

       [[0.56521739, 0.63197026, 0.175     , 0.31967213],
        [0.26086957, 0.40148699, 0.0875    , 0.24590164],
        [1.        , 0.18587361, 0.3375    , 0.30327869],
        [0.7826087 , 0.98141264, 0.6       , 0.44262295],
        [0.08695652, 0.39033457, 0.2125    , 0.19672131]],

       [[0.26086957, 0.40148699, 0.0875    , 0.24590164],
        [1.        , 0.18587361, 0.3375    , 0.30327869],
        [0.7826087 , 0.98141264, 0.6       , 0.44262295],
        [0.08695652, 0.39033457, 0.2125    , 0.19672131],
        [0.13043478, 0.27881041, 0.1875    , 0.28688525]],

       ...,

       [[0.13043478, 0.45724907, 0.2       , 0.2704918 ],
        [0.47826087, 0.62453532, 0.1375    , 0.196721

In [ ]:
y_train

array([[0.08837209],
       [0.05116279],
       [0.3372093 ],
       ...,
       [0.0627907 ],
       [0.18837209],
       [0.05581395]])

In [ ]:
model = Sequential()
model.add(GRU(16, input_shape=x_train[0].shape))
model.add(Dense(1,activation='linear'))

model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), metrics=['mae'])
model.summary()

early_stop = EarlyStopping(monitor='val_loss', patience=5)
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=100, batch_size=16, callbacks=[early_stop])

Model: "sequential_48"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_14 (GRU)                 (None, 16)                1056      
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 17        
Total params: 1,073
Trainable params: 1,073
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
79/79 [==============================] - 2s 8ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 2/100
79/79 [==============================] - 0s 4ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 3/100
79/79 [==============================] - 0s 4ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 4/100
79/79 [==============================] - 0s 4ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 5/100
79/79 [==================

In [ ]:
x_test = test[['hour', 'hour_bef_temperature','hour_bef_windspeed','hour_bef_ozone']]

In [ ]:
pred = model.predict(x_test)

ValueError: ignored